In [1]:
import pandas as pd
import torch.utils.data as data
import os
from sklearn.model_selection import train_test_split
import torch

In [54]:
# class FGDRDataset(data.Dataset):
    
#     def __init__(self):
        

In [17]:
fgadr_root_path = '/mnt/sda/haal02-data/FGADR-Seg-Set/Seg-set'
fgadr_csv_file_name = 'DR_Seg_Grading_Label.csv'
fgadr_image_path = '/mnt/sda/haal02-data/FGADR-Seg-Set/Seg-set/Original_Images'

data_file = pd.read_csv(os.path.join(fgadr_root_path, fgadr_csv_file_name), header=None, names=['image', 'level'])

name_list = []
label_list = []



for idx, row in data_file.iterrows():
    if os.path.isfile(os.path.join(fgadr_image_path, row[0])):
        name_list.append(os.path.join(fgadr_image_path, row[0]))
        label_list.append(row[1])
#         print(row[1])
    else:
        print(0)
# print(label_list)

In [3]:
int(0.85 * len(data_file))
# data_file.sort_values(by='level', ascending=True).level.value_counts()
fgdr_wt = data_file.level.value_counts().sort_index()

In [6]:
fgdr_wt

0    101
1    212
2    595
3    647
4    287
Name: level, dtype: int64

In [19]:
kaggle_data_path = '/mnt/sda/haal02-data/Kaggle-Retinopathy-Datasets/dr-data/trainLabels.csv'
kaggle_image_path = '/mnt/sda/haal02-data/Kaggle-Retinopathy-Datasets/dr-data/train/'
kaggle_df = pd.read_csv(kaggle_data_path, header=0)
kaggle_df.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [20]:
name_list = []
label_list = []
image_list = []
max_size = 6000
label_0_count = 0
for idx, row in kaggle_df.iterrows():
    file_name = row[0] + '.jpeg'
    if os.path.isfile(os.path.join(kaggle_image_path, file_name)):
        if row[1] == 0:
            if label_0_count < max_size:
                
                name_list.append(os.path.join(kaggle_image_path, file_name))
                label_list.append(row[1])
                image_list.append(row[0])
                label_0_count += 1
        else:
            name_list.append(os.path.join(kaggle_image_path, file_name))
            label_list.append(row[1])
            image_list.append(row[0])
#         print(row[1])
    else:
        print('Error')
# print(label_list)

In [21]:
kaggle_df.level.value_counts().sort_index()
# kaggle_df.level.value_counts().sort_values(axis=0, ascending=False)


0    25810
1     2443
2     5292
3      873
4      708
Name: level, dtype: int64

In [22]:
data = { 'image' : image_list,
           'level' : label_list}

csv_df = pd.DataFrame(data)
class_count = csv_df.level.value_counts().sort_index()
class_count

0    6000
1    2443
2    5292
3     873
4     708
Name: level, dtype: int64

In [23]:
weight_tensor = torch.zeros(1,5)
for idx, value in class_count.items():
    wt = value / len(csv_df)
#     print(len(data_file))
    weight_tensor[0][idx] = round((1 - wt), 2)

In [24]:
from sklearn.preprocessing import normalize
normalize(weight_tensor, norm='l1')

array([[0.15288221, 0.21052631, 0.16290726, 0.23558897, 0.23809524]])

In [21]:
print(int(0.7 * len(csv_df)))
print(int(0.85 * len(csv_df)))

10721
13018


In [8]:
X_train, X_test, y_train, y_test = train_test_split(name_list, label_list, test_size = 0.3)

In [24]:
# csv_df.to_csv('/mnt/sda/haal02-data/Kaggle-Retinopathy-Datasets/dr-data/truncated_trainLabels.csv', index=False)

In [25]:
csv_df

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
...,...,...
15311,44323_left,1
15312,44323_right,1
15313,44328_right,1
15314,44337_left,1
